In [164]:
import pandas as pd
# from gensim.models import Word2Vec
import tqdm
import ast
from collections import Counter
from navec import Navec
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, roc_auc_score, top_k_accuracy_score,\
                            f1_score, precision_score, recall_score, average_precision_score
from joblib import dump
import re

In [ ]:
df_train = pd.read_csv('final_markup/train.csv').sample(n=20000, random_state=999)
df_val = pd.read_csv('final_markup/val.csv').sample(n=5000, random_state=999)
df_test = pd.read_csv('final_markup/test.csv').sample(n=5000, random_state=999)

## Простые правила

Единественным правилом русского языка, которое точно определяет  необходимость знака припенания и которым мы можем с точностью воспользоваться - правило вводных слов.

Пробелы постараемся ставить после каждого слова, тк их как в трейне было много, так и вероятность получить прочий знак меньше, что потенциально должно улучшить метрики для базового алгоритма по правилам.

Что касается порчих символов:
- мы уверены, что точку нужно ставить в конце 
- больше никакие другие правила нам не помогут, поэтому придется экспрементировать


In [ ]:
# Возьмем вводные слова с сайта, чтобы руками не переносить под 400 слов

import requests
from bs4 import BeautifulSoup

target_url = 'https://russkiiyazyk.ru/sintaksis/spisok-vvodnyh-slov.html'
resp = requests.get(target_url)
print(resp)

soup = BeautifulSoup(resp.text, 'html.parser')
print(soup.title)

<Response [200]>
<title>Список вводных слов русского языка</title>


In [ ]:
all_comma_words_comb = soup.find_all("p")
commas_words_dict = {'smth':0}

for i in range(5,len(all_comma_words_comb)):
    if str(all_comma_words_comb[i])[:11] != '<p><strong>':
        for word_comb in all_comma_words_comb[i].text.replace(u'\xa0', u' ').split('\n'):
            try: 
                if word_comb[0] != '*' or word_comb[-1] != '*' or word_comb[0] != '\t' or word_comb[0] != "Δ" or word_comb != 'Поиск' or  word_comb[0] != '2':
                    commas_words_dict[word_comb]  = 1
            except:
                continue

check = list(commas_words_dict.keys())[-1]

while check != 'ясное дело':
    commas_words_dict.pop(check)
    check = list(commas_words_dict.keys())[-1]

# commas_words_dict - слова, которые мы будем отделять запятыми, тк они вводные 
commas_words_dict.pop('smth')

#before_commas_words_dict - попытка ввести деепричастные, причастные обороты и определенные суфиксы, перед которымы нужна запятая 

before_commas_words_dict = {}
before_commas_words_dict['а'] = 1
before_commas_words_dict['что'] = 1
before_commas_words_dict['когда'] = 1
before_commas_words_dict['который'] = 1
before_commas_words_dict['которая'] = 1
before_commas_words_dict['которое'] = 1
before_commas_words_dict['которые'] = 1
before_commas_words_dict['но'] = 1
before_commas_words_dict['ущ'] = 1
before_commas_words_dict['их'] = 1
before_commas_words_dict['ющ'] = 1
before_commas_words_dict['ую'] = 1
before_commas_words_dict['ащ'] = 1
before_commas_words_dict['ом'] = 1
before_commas_words_dict['ем'] = 1
before_commas_words_dict['вш'] = 1


Дальше продложено 2 алгоритма:
1) На основание только словарей выше и добавления точки в конце
2) Во втором берется вероятность возникновения определенного символо как в трейне (чтобы учесть ?, ! и тд, которые в правилах мы никак не описываем). Причем это дополнение 1 подхода, поэтому больший вес в пресказания вносят словари выше, поэтому и качество определения запятой просесть не должно 

In [343]:
import ast 
from scipy.stats import bernoulli
import random

class dumb_model():
    def __init__(self, data, type) -> list:
        self.data = data.text.apply(lambda x: x.split())
        self.type = type

        if type == 'vera': 
            punct_dict = {
                'total_words' : 0,
                ',' : 0,
                ':' : 0,
                ';' : 0,
                '.' : 0,
                '!' : 0,
                '?' : 0,
                '...' : 0
            }
            enctryption = {1:':',2:';',3:".",4:"!",5:"?",6:'...'}

            for lst in self.data:
                for ind in range(len(lst)):
                    if lst[ind][-1] in punct_dict:
                        punct_dict[lst[ind][-1]] += 1
                    punct_dict['total_words'] += 1 
            self.punct_dict = punct_dict   
            self.enctryption = enctryption
        
    def transform(self, string):
        string = ast.literal_eval(string)
        if self.type == 'dict':
            res = ['']
            for i in range(len(string)):
                if commas_words_dict.get(string[i], 0) == 1 and i == 0:
                    res.append(string[i] + ",")
                elif commas_words_dict.get(string[i], 0) == 1 and i != 0:
                    previous = res.pop()
                    res.append(previous+",")
                    res.append(string[i]+",")
                else:
                    if before_commas_words_dict.get(string[i], 0) == 1 and i != 0:
                        previous = res.pop()
                        res.append(previous+",")
                        res.append(string[i])
                    elif len(string[i]) > 2:
                        if before_commas_words_dict.get(string[i][-2:], 0) == 1 and string[i][-2:] != string[i]:
                            previous = res.pop()
                            res.append(previous+",")
                            res.append(string[i])
                        elif string[i] == 'г':
                            res.append(string[i]+".")
                        else:
                            res.append(string[i])
                    else:
                        res.append(string[i])
            res = " ".join(res)
            if res[-1] in ',;:':
                return res[1:-1]+'.'
            else:
                return res[1:] +'.'
        
        elif self.type == 'vera':
            res = ''
            for i in range(len(string)):
                flag_dict = {',':0, '.':0, ':':0, ';':0, '?':0, '!':0, 'before_,':0}
                if commas_words_dict.get(string[i], 0) == 1 and i == 0:
                    flag_dict[','] += 1
                elif commas_words_dict.get(string[i], 0) == 1 and i != 0:
                    flag_dict['before_,'] += 1
                    flag_dict[','] += 1
                else:
                    if before_commas_words_dict.get(string[i], 0) == 1 and i != 0:
                        flag_dict['before_,'] += 1
                    elif len(string[i]) > 2:
                        if before_commas_words_dict.get(string[i][-2:], 0) == 1 and string[i][-2:] != string[i]:
                            flag_dict['before_,'] += 1
                        if string[i] == 'г':
                            flag_dict['.'] += 1
                right = ''
                left = ' '

                if bernoulli.rvs(size=1,p=(sum(self.punct_dict.values()) - self.punct_dict['total_words'])/self.punct_dict['total_words'])[0] == 1 or sum(flag_dict.values()) > 1:
                    if  flag_dict['before_,'] > 0:
                        left = ', '
                    if bernoulli.rvs(size=1,p=self.punct_dict[',']/self.punct_dict['total_words'])[0] == 1 or flag_dict[','] > 0:
                        right = ','
                    else:
                        mark = self.enctryption.get(random.choices(list(self.enctryption.keys()), weights=list(self.punct_dict.values())[2:], k=1)[0], '')
                        if mark == '.' or flag_dict['.'] > 0:
                            right = '.'
                        else:
                            right = mark
                res += left + string[i] + right
            if res[0] == ' ':
                res = res[1:]
            if res[-1] in ',:;':
                return res[:-1] + self.enctryption.get(random.choices(list(self.enctryption.keys())[2:], weights=list(self.punct_dict.values())[4:], k=1)[0], '.')
            else:
                return res + '.'

In [344]:
print("Original text       : ", df_test['text'].iloc[0])
print("Probability aproach : ", dumb_model(df_train, type = 'vera').transform(df_test['tokens'].iloc[0]))
print("Dictionary aproach  : ", dumb_model(df_train, type = 'dict').transform(df_test['tokens'].iloc[0]))

Original text       :  Теперь известные как Алая Ведьма и Ртуть, Ванда и Пьетро сопровождали Магнето и стали его верными подручными мутантами в его поисках могущества, сталкиваясь с героями Людьми Икс. Спустя некоторое время брат и сестра, разочарованные в Магнето и чувствуя, что их долг благодарности уже был вдоволь заплачен, решили расстаться с их прежними террористическими действиями; вместе с лучником и бывшим преступником Соколиным Глазом они присоединились к Мстителям под предводительством Капитана Америки. Как член этой четвёрки, первой реорганизации всё ещё молодой команды супергероев, Алая Ведьма безупречно сражалась против Доктора Дума, Человека-крота, Канга и других.
Probability aproach :  теперь известные как: алая ведьма и ртуть. ванда. и пьетро сопровождали магнето. и. стали его верными, подручными мутантами в его поисках могущества сталкиваясь с героями людьми икс. спустя некоторое время брат и сестра. разочарованные. в магнето и чувствуя. что их долг. благодарности уже 

Подход через словари хорошо определяет запятые, но он не учитывает сложные предложения, тк в нем не закладывалось определение подлежащего и сказуемого. Однако это самый простой подъод и давайте прогоним его по тесту

In [369]:
def get_data_for_metrics(df_train, df_test):
    model_pr = dumb_model(df_train, type = 'vera')
    model_dc = dumb_model(df_train, type = 'dict')

    df_test['model_probs_pred'] = df_test.tokens.apply(lambda x: model_pr.transform(x))
    df_test['model_dict_pred'] = df_test.tokens.apply(lambda x: model_dc.transform(x))

    enctryption = {0:",",1:':',2:';',3:".",4:"!",5:"?",6:'...'}
    back_enctryption = {",":0,':':1,';':2,".":3,"!":4,"?":5,'...':6,}
    
    def transform_to_labels(data, back_enctryption):
        output = []
        for sentence in data:
            for word in sentence.split():
                if word != "":
                    if word[-1] in back_enctryption:
                        output.append(back_enctryption[word[-1]])
                    else:
                        output.append(7)
        return np.array(output)
    
    
    return transform_to_labels(df_test.text, back_enctryption), transform_to_labels(df_test['model_probs_pred'], back_enctryption), transform_to_labels(df_test['model_dict_pred'], back_enctryption)
            
    

In [370]:
y_true, y_pred_prob, y_pred = get_data_for_metrics(df_train, df_test)

In [371]:
len(y_true), len(y_pred_prob), len(y_pred)

(211021, 211021, 211021)

In [413]:
def calc_metrics(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    print('Доля пробелов:', (y_true == 7).mean())
    metrics = []
    enctryption = {0:",",1:':',2:';',3:".",4:"!",5:"?",6:'...', 7:'o'}
    sorted_dict = {enctryption[i[0]] : i[1] for i in sorted(Counter(y_true).items())}
    
    metrics.append(sorted_dict.values())
    metrics.append(f1_score(y_true, y_pred, average=None))
    metrics.append(precision_score(y_true, y_pred, average=None, zero_division=0))
    metrics.append(recall_score(y_true, y_pred, average=None, zero_division=0))
    metrics_index = ['Count', 'F1-Score', 'Precision', 'Recall']
    df_metrics = pd.DataFrame(metrics, columns=sorted_dict.keys(), index=metrics_index)
    
    return df_metrics

In [414]:
# Via dictionary aproach

calc_metrics(y_true, y_pred)

Доля пробелов: 0.851773046284493


,",",:,;,.,!,?,o
Count,15674.000000,560.0,510.0,14458.000000,30.0,47.0,179742.000000
F1-Score,0.076916,0.0,0.0,0.033200,0.0,0.0,0.874524
Precision,0.075068,0.0,0.0,0.064600,0.0,0.0,0.851885
Recall,0.078857,0.0,0.0,0.022341,0.0,0.0,0.898399


In [415]:
# Via probability aproach

calc_metrics(y_true, y_pred_prob)

Доля пробелов: 0.851773046284493


,",",:,;,.,!,?,o
Count,15674.000000,560.000000,510.000000,14458.000000,30.0,47.0,179742.000000
F1-Score,0.042103,0.002497,0.003044,0.096942,0.0,0.0,0.831920
Precision,0.070655,0.001919,0.002488,0.071174,0.0,0.0,0.851921
Recall,0.029986,0.003571,0.003922,0.151957,0.0,0.0,0.812837


Итого: 

- F1-Score как и предполагалось в определение пробелов больше в подходе без вероятностей, тк мы захломляем текст ненужными знаками
- Однако только в вероятностном подходе мы как-то учитываем прочие знаки препинание (помимо точки и запятой)
- Подходы конечно не айс, но F1-Score запятой побили в сравнение с лог регрессией) Остальные метрики победить не смогли, но мы не останавливаемся

